# Missouri Data Preparation Tutorial
Jose Hernandez  
10/10/2016   
CRPE   

### Note:
This is for illustration and teaching purposes.

In [1]:
rm(list=ls())
suppressMessages(library(dplyr))

#### Load the Data

In [2]:
file <- "~/Google Drive/CRPE/KauffmanKC/Data/MAP_School_Final.csv" 
data <- read.csv(file, header=TRUE,stringsAsFactors = F,fileEncoding="latin1")

In [3]:
names(data)

[1] "COUNTY_DISTRICT_SCHOOL_CODE" "COUNTY_DISTRICT"            
 [3] "DISTRICT_NAME"               "SCHOOL_CODE_0001"           
 [5] "SCHOOL_NAME"                 "CATEGORY"                   
 [7] "TYPE"                        "CONTENT_AREA"               
 [9] "GRADE_LEVEL"                 "YEAR"                       
[11] "ACCOUNTABLE"                 "PARTICIPANT"                
[13] "REPORTABLE"                  "LEVEL_NOT_DETERMINED"       
[15] "LEVEL_NOT_DETERMINED_PCT"    "BELOW_BASIC"                
[17] "BELOW_BASIC_PCT"             "BASIC"                      
[19] "BASIC_PCT"                   "PROFICIENT"                 
[21] "PROFICIENT_PCT"              "ADVANCED"                   
[23] "ADVANCED_PCT"                "BOTTOM_TWO_LEVELS"          
[25] "BOTTOM_TWO_LEVELS_PCT"       "TOP_TWO_LEVELS"             
[27] "TOP_TWO_LEVELS_PCT"          "MAP_INDEX"                  
[29] "MEAN_SCALE_SCORE"            "MEDIAN_SCALE_SCORE"         
[31] "MEDIAN_TERRANOVA"

In [4]:
#summary of all the columns 
summary(data)

 COUNTY_DISTRICT_SCHOOL_CODE COUNTY_DISTRICT  DISTRICT_NAME     
 Min.   :1.090e+07           Min.   :  1090   Length:141253     
 1st Qu.:3.613e+08           1st Qu.: 36126   Class :character  
 Median :5.311e+08           Median : 53113   Mode  :character  
 Mean   :6.129e+08           Mean   : 61287                     
 3rd Qu.:9.609e+08           3rd Qu.: 96088                     
 Max.   :4.014e+09           Max.   :401401                     
                                                                
 SCHOOL_CODE_0001 SCHOOL_NAME          CATEGORY             TYPE          
 Min.   :1015     Length:141253      Length:141253      Length:141253     
 1st Qu.:1050     Class :character   Class :character   Class :character  
 Median :4020     Mode  :character   Mode  :character   Mode  :character  
 Mean   :3132                                                             
 3rd Qu.:4060                                                             
 Max.   :9990                 

Things look neormal i.e. numbers are numbers...   
This Data has multiple categories by year, we want to issolate subject proficiency scores by school and year.

In [5]:
#look at the cases per year 2010-2014
table(data$YEAR)
#look at the subset categories FRL, subgroup, etc.
table(data$CATEGORY)
#We only want to look at school totals 


 2010  2011  2012  2013  2014 
28514 28172 28160 28217 28190 


       MSIP5 Total MSIP5 Total wo EOC              Total 
             32402              32145              76706 

In [6]:
data <- filter(data, CATEGORY == "Total")

In [7]:
head(data)

COUNTY_DISTRICT_SCHOOL_CODE,COUNTY_DISTRICT,DISTRICT_NAME,SCHOOL_CODE_0001,SCHOOL_NAME,CATEGORY,TYPE,CONTENT_AREA,GRADE_LEVEL,YEAR,⋯,ADVANCED,ADVANCED_PCT,BOTTOM_TWO_LEVELS,BOTTOM_TWO_LEVELS_PCT,TOP_TWO_LEVELS,TOP_TWO_LEVELS_PCT,MAP_INDEX,MEAN_SCALE_SCORE,MEDIAN_SCALE_SCORE,MEDIAN_TERRANOVA
10901050,1090,ADAIR CO. R-I,1050,ADAIR CO. HIGH,Total,Total,Eng. Language Arts,7,2010,⋯,0,0.0,17,70.8,7,29.2,NA,657.6,663.5,55.5
10901050,1090,ADAIR CO. R-I,1050,ADAIR CO. HIGH,Total,Total,Eng. Language Arts,7,2011,⋯,3,14.3,10,47.6,11,52.4,NA,680.5,683.5,68.5
10901050,1090,ADAIR CO. R-I,1050,ADAIR CO. HIGH,Total,Total,Eng. Language Arts,7,2012,⋯,2,11.1,10,55.6,8,44.4,NA,674.1,675.0,65.0
10901050,1090,ADAIR CO. R-I,1050,ADAIR CO. HIGH,Total,Total,Eng. Language Arts,7,2013,⋯,4,25.0,8,50.0,8,50.0,NA,684.5,680.0,70.0
10901050,1090,ADAIR CO. R-I,1050,ADAIR CO. HIGH,Total,Total,Eng. Language Arts,7,2014,⋯,4,15.4,10,38.5,16,61.5,NA,684.1,686.0,70.0
10901050,1090,ADAIR CO. R-I,1050,ADAIR CO. HIGH,Total,Total,Eng. Language Arts,8,2010,⋯,2,7.1,14,50.0,14,50.0,NA,691.3,696.0,61.5


Notice that we have grade specific categories X Content_Area X Year for each school.  We want school specific categories X year.  We will need to aggregate the grade level infomration across proficiency categories.  

In [8]:
test1 <- data %>%
  group_by(SCHOOL_CODE_0001,SCHOOL_NAME,CONTENT_AREA,YEAR) %>%
  summarise(number = n(),
            REPORTED = sum(REPORTABLE,na.rm=T))

Let's test out aggregation using dplyr   
I'm combining content areas by school   
In the case where there are multiple grades of the content area I want to either:   
1. Add the counts of students 
2. Average out the percentages (not ideal but can regain percentages from aggregate counts in this case)
This is what the previous code did with "REPORTABLE" or the number of students that they can report took the test. 

In [9]:
filter(test1, SCHOOL_NAME == 'ADAIR CO. HIGH' & CONTENT_AREA=='Eng. Language Arts' & YEAR == 2010)

SCHOOL_CODE_0001,SCHOOL_NAME,CONTENT_AREA,YEAR,number,REPORTED
1050,ADAIR CO. HIGH,Eng. Language Arts,2010,8,169


The "number" indicates the # of elements aggregated for this [school X subject X grade] and the variable "REPORTED"(I know not ideal) is the sum of the counts of students reported.  Let's see if this checks out.

In [10]:
#Filter the original data to give you that school and subject and display the counts that we aggregated
test2 <- filter(data, SCHOOL_NAME == 'ADAIR CO. HIGH' & CONTENT_AREA=='Eng. Language Arts' & YEAR == 2010)

test2$REPORTABLE

[1] 24 28 17 26 17 20 17 20

In [11]:
24+28+17+26+17+20+17+20

[1] 169

#### Victory!
Let's do it for all the columns of interest.

In [12]:
MO_data = data %>%
            group_by(SCHOOL_CODE_0001,SCHOOL_NAME,CONTENT_AREA,YEAR) %>%
              summarise(number = n(),
              enrolled = sum(ACCOUNTABLE, na.rm=T), 
              participant = sum(PARTICIPANT, na.rm=T),                
              reported = sum(REPORTABLE, na.rm=T),
              level_unknown = sum(LEVEL_NOT_DETERMINED, na.rm=T),
              level_unknown_pct = mean(LEVEL_NOT_DETERMINED_PCT, na.rm=T),   
              below_basic = sum(BELOW_BASIC, na.rm=T),
              below_basic_pct = mean(BELOW_BASIC_PCT, na.rm=T), 
              basic = sum(BASIC, na.rm=T),                      
              basic_pct = mean(BASIC_PCT, na.rm=T),
              proficent = sum(PROFICIENT, na.rm=T),
              proficient_pct = mean(PROFICIENT_PCT, na.rm=T),             
              advanced = sum(ADVANCED, na.rm=T),
              advanced_pct = mean(ADVANCED_PCT, na.rm=T),
              bottom_two = sum(BOTTOM_TWO_LEVELS, na.rm=T),          
              bottom_two_pct = mean(BOTTOM_TWO_LEVELS_PCT, na.rm=T),
              top_two = sum(TOP_TWO_LEVELS, na.rm=T),
              top_two_pct = mean(TOP_TWO_LEVELS_PCT, na.rm=T),         
              #throw away MAP_INDEX --> NAs
              mean_scale_score = mean(MEAN_SCALE_SCORE, na.rm=T),
              median_scale_score = mean(MEDIAN_SCALE_SCORE, na.rm=T),         
              median_terra = mean(MEDIAN_TERRANOVA, na.rm=T)
              )

In [13]:
head(MO_data)

SCHOOL_CODE_0001,SCHOOL_NAME,CONTENT_AREA,YEAR,number,enrolled,participant,reported,level_unknown,level_unknown_pct,⋯,proficient_pct,advanced,advanced_pct,bottom_two,bottom_two_pct,top_two,top_two_pct,mean_scale_score,median_scale_score,median_terra
1015,HILLTOP SCHOOL,Eng. Language Arts,2010,1,11,0,11,0,0,⋯,0.0,0,0.0,11,100.0,0,0.0,652.5,659.0,27
1015,JUVENILE DETENTION CTR.,Eng. Language Arts,2010,1,13,0,13,0,0,⋯,38.5,0,0.0,8,61.5,5,38.5,196.8,197.0,NaN
1015,JUVENILE DETENTION CTR.,Eng. Language Arts,2013,1,12,0,12,0,0,⋯,58.3,2,16.7,3,25.0,9,75.0,202.7,200.0,NaN
1015,JUVENILE DETENTION CTR.,Mathematics,2010,1,19,0,19,0,0,⋯,5.3,1,5.3,17,89.5,2,10.5,181.6,177.0,NaN
1015,JUVENILE DETENTION CTR.,Mathematics,2011,1,19,0,19,0,0,⋯,31.6,2,10.5,11,57.9,8,42.1,197.2,193.0,NaN
1015,JUVENILE DETENTION CTR.,Mathematics,2013,1,16,0,16,0,0,⋯,50.0,0,0.0,8,50.0,8,50.0,197.3,198.5,NaN


Getting closer...  